# `Firestore`

using google-cloud-firestore and firebase_admin to verify credentials

In [32]:
import firebase_admin
from firebase_admin import firestore

In [52]:
import os
print(os.environ.get("GOOGLE_CLOUD_PROJECT"))

johannesvc


In [50]:
# os.environ["GOOGLE_CLOUD_PROJECT"] = 'johannesvc'

or in command prompt:

setx GOOGLE_APPLICATION_CREDENTIALS "C:/Users/johan/Documents/GitHub/johannes.vc/google_credentials.json"

In [33]:
from firebase_admin import credentials

cred = credentials.Certificate(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

# and then the same
app = firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

There's also an `ApplicationDefault` (using an existing credential)

In [ ]:
# default
cred = credentials.ApplicationDefault()

and if the GOOGLE_APPLICATION_CREDENTIALS environment variable is set to the path of your service account key JSON file

In [35]:
app = firebase_admin.initialize_app() # (cred)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
db = firestore.client()
db.project

## Create a new collection and document

In [3]:
test_doc = db.collection("users").document("alovelace")

set fields (much like insert_one - $set in mongodb)

In [4]:
test_doc.set({"first": "Ada", "last": "Lovelace", "born": 1815})

update_time {
  seconds: 1697618452
  nanos: 574522000
}

insert without document id

In [15]:
city = {"name": "Tokyo", "country": "Japan"}
db.collection("cities").add(city)

(DatetimeWithNanoseconds(2023, 10, 18, 8, 57, 2, 22559, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x241dbf093f0>)

In [22]:
# make doc without setting an id
new_city_ref = db.collection("cities").document()

# later...
new_city_ref.set({"name": "Gent"})

update_time {
  seconds: 1697619718
  nanos: 990470000
}

.set overwrites unless merge=True

In [24]:
new_city_ref.set({"capital": True}, merge=True) # otherwise it overwrites

update_time {
  seconds: 1697619787
  nanos: 790536000
}

## update

In [25]:
new_city_ref.update({"capital": True})

update_time {
  seconds: 1697619787
  nanos: 790536000
}

In [21]:
# add a timestamp
new_city_ref.update({"timestamp": firestore.SERVER_TIMESTAMP})

update_time {
  seconds: 1697619648
  nanos: 310000
}
transform_results {
  timestamp_value {
    seconds: 1697619647
    nanos: 720000000
  }
}

nested objects can be reference with dot notation

note: `favorites.color`

In [34]:
# Create an initial document to update
frank_ref = db.collection("users").document("frank")
frank_ref.set(
    {
        "name": "Frank",
        "favorites": {"food": "Pizza", "color": "Blue", "subject": "Recess"},
        "age": 12,
    }
)

# Update age and favorite color
frank_ref.update({"age": 13, "favorites.color": "Red"})

update_time {
  seconds: 1697620254
  nanos: 696226000
}

delete

In [ ]:
# delete doc
db.collection("cities").document("DC").delete()

In [ ]:
# delete field
city_ref = db.collection("cities").document("BJ")
city_ref.update({"capital": firestore.DELETE_FIELD})

find / read docs

In [37]:
doc_ref = db.collection("users").document("frank").get()

print(doc_ref.to_dict())

{'age': 13, 'favorites': {'food': 'Pizza', 'subject': 'Recess', 'color': 'Red'}, 'name': 'Frank'}


In [31]:
docs = db.collection("cities").list_documents()

for doc in docs:
    print(doc.get().to_dict())

{'capital': True, 'timestamp': DatetimeWithNanoseconds(2023, 10, 18, 9, 0, 47, 720000, tzinfo=datetime.timezone.utc), 'name': 'Gent'}
{'capital': True, 'name': 'Gent'}
{'name': 'Tokyo', 'country': 'Japan'}


stream is better than `.list_documents()` then `.get()`

In [10]:
docs = db.collection("users").stream()

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

alovelace => {'last': 'Lovelace', 'first': 'Ada', 'born': 1815}


It allows filters

In [38]:
docs = (
    db.collection("cities").where(filter=firestore.FieldFilter("capital", "==", True)).stream())

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

myPDuz1Mio0QkN6terXB => {'capital': True, 'timestamp': DatetimeWithNanoseconds(2023, 10, 18, 9, 0, 47, 720000, tzinfo=datetime.timezone.utc), 'name': 'Gent'}
ullBnrY1xFR9ub30ef1i => {'capital': True, 'name': 'Gent'}


In [43]:
cities_ref = db.collection("cities")

query = cities_ref.where(
    filter=firestore.FieldFilter("name", "==", "Gent")
    )
query.

subcollections

In [41]:
db.collection("users").document("Frank").listCollections()

AttributeError: 'DocumentReference' object has no attribute 'listCollections'

In [39]:
collections = db.collection("users").document("Frank").collections()
for collection in collections:
    for doc in collection.stream():
        print(f"{doc.id} => {doc.to_dict()}")

# Use Google secrets manager

In [8]:
from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload for the given secret version if one exists. 
    The version can be a version number as a string (e.g. "5") or an
    alias (e.g. "latest").
    """
    client = secretmanager.SecretManagerServiceClient.from_service_account_json('C:/Users/johan/Documents/GitHub/johannes.vc/google_credentials.json')

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})
    
    # Return the decoded payload.
    return response.payload.data.decode('UTF-8')

import firebase_admin
from firebase_admin import credentials, firestore
import json

# Retrieve JSON credentials from Secret Manager
project_id = "62117585564"
secret_id = "GOOGLE_APPLICATION_CREDENTIALS"
version_id = 'latest'  # or a specific version

json_credentials = access_secret_version(project_id, secret_id, version_id)
parsed_credentials = json.loads(json_credentials)

# Initialize the Firebase app
cred = credentials.Certificate(parsed_credentials)
app = firebase_admin.initialize_app(cred)

# Initialize Firestore
db = firestore.client()

# to transfer from mongodb

In [27]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import os

uri = os.getenv('MONGODB_KEY')

client = MongoClient(uri)

db = client.flask_tutorial

In [29]:
db.blogs.find_one().keys()

dict_keys(['_id', 'username', 'title', 'body', 'author_id', 'created'])

# Add blogs

In [56]:
db.collection('blogs').add({'id': firestore.SERVER_TIMESTAMP,
                        'username': 'joahnnes',
                        'title': 'blog', 
                        'body': 'blablaba', 
                        'author_id': None,
                        'created': firestore.SERVER_TIMESTAMP}) 

(DatetimeWithNanoseconds(2023, 10, 18, 12, 10, 24, 65364, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x26b15987fa0>)

In [57]:
posts = db.collection('blogs').order_by('id', 
                                        direction=firestore.Query.DESCENDING
                                        ).stream()

In [58]:
[post.to_dict() for post in posts]

[{'title': 'blog',
  'created': DatetimeWithNanoseconds(2023, 10, 18, 12, 10, 24, 24000, tzinfo=datetime.timezone.utc),
  'id': DatetimeWithNanoseconds(2023, 10, 18, 12, 10, 24, 24000, tzinfo=datetime.timezone.utc),
  'body': 'blablaba',
  'author_id': None,
  'username': 'joahnnes'}]